In [ ]:
import numpy as np
import cv2
import os
import open3d as o3d
import matplotlib.pyplot as plt

# Directory where data is saved
output_dir = 'sensor_data'

# Load saved data
def load_data(frame_number):
    lidar_file = os.path.join(output_dir, f'lidar_{frame_number}.npy')
    camera_file = os.path.join(output_dir, f'camera_{frame_number}.npy')
    
    if not os.path.exists(lidar_file) or not os.path.exists(camera_file):
        raise FileNotFoundError(f"Files for frame {frame_number} not found.")
    
    lidar_data = np.load(lidar_file)
    camera_data = np.load(camera_file)
    return lidar_data, camera_data

# Initialize visual odometry
orb = cv2.ORB_create()
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
trajectory = []

# Initialize Open3D visualizer
vis = o3d.visualization.Visualizer()
vis.create_window()

# Create Open3D point cloud object
point_cloud = o3d.geometry.PointCloud()

# Process each frame based on availability
frame_numbers = sorted(set(int(f.split('_')[1].split('.')[0]) for f in os.listdir(output_dir) if f.startswith('lidar_')))
prev_img = None
prev_kp = None
prev_des = None
position = np.array([0, 0, 0], dtype=np.float32)  

for frame_number in frame_numbers:
    try:
        lidar_data, camera_data = load_data(frame_number)
    except FileNotFoundError:
        continue  # Skip if not found

    # Add LIDAR data to point cloud
    point_cloud.points = o3d.utility.Vector3dVector(lidar_data)
    
    # Convert camera data to grayscale 
    current_img = cv2.cvtColor(camera_data, cv2.COLOR_BGR2GRAY)
    current_kp, current_des = orb.detectAndCompute(current_img, None)

    if prev_img is not None:
        # Match previous and current image features
        matches = bf.match(prev_des, current_des)
        matches = sorted(matches, key=lambda x: x.distance)
        
        # Extract location of matched keypoints
        prev_pts = np.float32([prev_kp[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
        curr_pts = np.float32([current_kp[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
        
        # Estimate motion of the camera
        E, mask = cv2.findEssentialMat(curr_pts, prev_pts, focal=1.0, pp=(0, 0), method=cv2.RANSAC, prob=0.999, threshold=1.0)
        _, R, t, mask = cv2.recoverPose(E, curr_pts, prev_pts)
        
        # Update pos
        scale = 1.0  
        position += scale * t.flatten()

        # Append new pos to trajectory
        trajectory.append(position.copy())

    # Update previous image and keypoints
    prev_img = current_img
    prev_kp = current_kp
    prev_des = current_des

# Convert trajectory to Open3D format
trajectory_points = np.array(trajectory)
trajectory_lines = [[i, i + 1] for i in range(len(trajectory_points) - 1)]
colors = [[1, 0, 0] for _ in range(len(trajectory_lines))]  

trajectory_line_set = o3d.geometry.LineSet()
trajectory_line_set.points = o3d.utility.Vector3dVector(trajectory_points)
trajectory_line_set.lines = o3d.utility.Vector2iVector(trajectory_lines)
trajectory_line_set.colors = o3d.utility.Vector3dVector(colors)

# Add point cloud and trajectory to visualizer
vis.add_geometry(point_cloud)
vis.add_geometry(trajectory_line_set)

# Run visualizer
vis.run()
vis.destroy_window()

# Plot final trajectory
plt.plot(trajectory_points[:, 0], trajectory_points[:, 1], '-o')
plt.title('Trajectory')
plt.xlabel('X')
plt.ylabel('Y')
plt.show()
